# **DATA PROCESSING & EDA**

# **0.SETUP**

In [1]:
# ==============================================================================
# Step 0: 环境配置与路径 (Setup)
# ==============================================================================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 挂载 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ---------------------------------------------------------
# ⭐ 关键点：定义 BASE_PATH
# ---------------------------------------------------------
BASE_PATH = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT'

# 定义输入和输出的具体路径
RAW_DATA_PATH = os.path.join(BASE_PATH, 'data/raw/Student_Depression_Dataset.csv')
PROCESSED_DATA_DIR = os.path.join(BASE_PATH, 'data/processed')

# 自动创建输出文件夹（如果不存在的话），防止报错
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

print(f"📂 工作目录设置完成。")
print(f"📥 读取路径: {RAW_DATA_PATH}")
print(f"📤 输出路径: {PROCESSED_DATA_DIR}")

Mounted at /content/drive
📂 工作目录设置完成。
📥 读取路径: /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/raw/Student_Depression_Dataset.csv
📤 输出路径: /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/processed


# **1.LOAD & INSPECT**

In [2]:
# ==============================================================================
# Step 1: 数据加载与初探 (Load & Inspect)
# ==============================================================================
# 读取数据
df = pd.read_csv(RAW_DATA_PATH)

#原始数据备份
df_raw = pd.read_csv(RAW_DATA_PATH)

print("\n--- 原始数据概览 ---")
print(f"数据形状: {df.shape}")
print(df.info())


--- 原始数据概览 ---
数据形状: (27901, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27901 entries, 0 to 27900
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     27901 non-null  int64  
 1   Gender                                 27901 non-null  object 
 2   Age                                    27901 non-null  float64
 3   City                                   27901 non-null  object 
 4   Profession                             27901 non-null  object 
 5   Academic Pressure                      27901 non-null  float64
 6   Work Pressure                          27901 non-null  float64
 7   CGPA                                   27901 non-null  float64
 8   Study Satisfaction                     27901 non-null  float64
 9   Job Satisfaction                       27901 non-null  float64
 10  Sleep Duration                      

**Fig. Raw Data Diagnosis**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
from matplotlib.patches import Patch

# ==========================================
# 1. 数据准备 (Data Preparation)
# ==========================================
df_plot = df.copy()
df_plot.columns = df_plot.columns.str.strip()

# 1.1 基础过滤 (Basic Trimming)
# 即使是 Raw Data，极端的脏数据（如Age=0）也可以先滤掉，
# 或者保留它们以在图中展示“异常”，取决于你想不想让图好看点。
# 这里我们保留稍微宽松一点的范围，体现 "Raw" 的感觉，但去掉极值以免压扁图形
df_plot = df_plot[(df_plot['Age'] >= 17) & (df_plot['Age'] <= 35)]
if 'CGPA' in df_plot.columns:
    df_plot = df_plot[df_plot['CGPA'] > 3.0]

# 1.2 Top N 处理 (简化高基数类别)
high_cardinality_cols = {'City': 15, 'Profession': 5, 'Degree': 15}
plot_cols = []
for col, top_n in high_cardinality_cols.items():
    if col in df_plot.columns:
        top_cats = df_plot[col].value_counts().nlargest(top_n).index
        new_col = f'{col}_Simple'
        df_plot[new_col] = df_plot[col].apply(lambda x: x if x in top_cats else 'Others')
        plot_cols.append(new_col)

# 1.3 筛选特征
# 保留 'Job Satisfaction' 等用于证明“零膨胀”的特征
cols_to_drop = ['id', 'ID', 'City', 'Profession', 'Degree', 'Depression']
target_col = 'Depression'
features_numeric = [c for c in df_plot.select_dtypes(include=['number']).columns if c not in cols_to_drop and c != target_col]
features_categorical = [c for c in df_plot.select_dtypes(include=['object', 'category']).columns if c not in cols_to_drop and c != target_col]
all_features = sorted(list(set(features_numeric + features_categorical + plot_cols)))

print(f"🚀 [Raw Data Diagnosis] 正在绘制 {len(all_features)} 个特征...")

# ==========================================
# 2. 绘图配置 (深色本色边框)
# ==========================================
# 填充色
COLOR_LIME = "#8FD94D"
COLOR_SAKURA = "#FF5588"
# 边框色 (Deep Tone-on-Tone)
EDGE_LIME = "#8FD94D"
EDGE_SAKURA = "#FF5588"

n_cols = 4
n_rows = math.ceil(len(all_features) / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5 * n_rows), dpi=300)
axes = axes.flatten()

# ==========================================
# 3. 循环绘图
# ==========================================
for i, feature in enumerate(all_features):
    ax = axes[i]
    is_numeric = feature in features_numeric

    # 动态判断 KDE
    draw_kde = is_numeric and 'Satisfaction' not in feature
    if draw_kde:
        for dep_status in df_plot[target_col].unique():
            subset = df_plot[df_plot[target_col] == dep_status][feature].dropna()
            if subset.nunique() < 2 or subset.var() < 1e-9:
                draw_kde = False; break

    data_h = df_plot[df_plot[target_col] == 0]
    data_d = df_plot[df_plot[target_col] == 1]

    # 统一 Bins
    computed_bins = None
    if is_numeric:
        val_min = df_plot[feature].min()
        val_max = df_plot[feature].max()
        if pd.notnull(val_min) and pd.notnull(val_max):
            computed_bins = np.linspace(val_min, val_max, 16)

    # 分层绘制
    sns.histplot(
        data=data_h, x=feature,
        color=COLOR_LIME, edgecolor=EDGE_LIME, linewidth=0.8, alpha=0.2,
        bins=computed_bins, discrete=(not is_numeric), shrink=0.8, element="bars",
        stat="percent", ax=ax, kde=draw_kde, line_kws={'color': EDGE_LIME, 'linewidth': 0.8}
    )

    sns.histplot(
        data=data_d, x=feature,
        color=COLOR_SAKURA, edgecolor=EDGE_SAKURA, linewidth=0.8, alpha=0.2,
        bins=computed_bins, discrete=(not is_numeric), shrink=0.8, element="bars",
        stat="percent", ax=ax, kde=draw_kde, line_kws={'color': EDGE_SAKURA, 'linewidth': 0.8}
    )

    # 装饰
    clean_title = feature.replace('_Simple', ' (Top)').replace('Satisfaction', 'Sat.')
    ax.set_title(clean_title, fontweight='bold', fontsize=11)
    ax.set_xlabel("")
    ax.set_ylabel("Percent" if i % n_cols == 0 else "")

    if not is_numeric:
        ax.tick_params(axis='x', rotation=45)
        plt.setp(ax.get_xticklabels(), fontsize=8, ha="right", rotation_mode="anchor")

    if 'Satisfaction' in feature or 'Pressure' in feature:
        ax.xaxis.get_major_locator().set_params(integer=True)

    if ax.get_legend(): ax.get_legend().remove()

# ==========================================
# 4. 收尾
# ==========================================
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

# 【核心修改】标题更改为 Raw Data Diagnosis
fig.suptitle(
    "Raw Data Diagnosis: Initial Feature Distributions & Anomalies",
    fontsize=20, fontweight='bold', y=0.96
)

legend_elements = [
    Patch(facecolor=COLOR_LIME, edgecolor=EDGE_LIME, linewidth=1.5, alpha=0.3, label='Healthy (0)'),
    Patch(facecolor=COLOR_SAKURA, edgecolor=EDGE_SAKURA, linewidth=1.5, alpha=0.3, label='Depressed (1)'),
]
fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.95, 0.96), frameon=False)

plt.tight_layout()
plt.subplots_adjust(top=0.92, hspace=0.6, wspace=0.2)

plt.savefig('raw_data_diagnosis.png', dpi=300, bbox_inches='tight')
plt.show()

🚀 [Raw Data Diagnosis] 正在绘制 16 个特征...


Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x7e6261dd6d40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/transforms.py", line 195, in <lambda>
    self, lambda _, pop=child._parents.pop, k=id_self: pop(k))
                                                       ^^^^^^
KeyboardInterrupt: 


KeyboardInterrupt: 

# **2. DATA CLEANING**

In [3]:
import pandas as pd
import numpy as np

# ==============================================================================
# Step 2: 数据清洗 (Data Cleaning & Preprocessing)
# ==============================================================================
print("\n--- 开始数据清洗 ---")
print(f"初始数据形状: {df.shape}")

# -----------------------------------------------------------
# 1. 删除无关列 (Drop ID)
# -----------------------------------------------------------
if 'id' in df.columns:
    df = df.drop(columns=['id'])
elif 'ID' in df.columns:
    df = df.drop(columns=['ID'])
print("✅ 已删除 ID 列")

# -----------------------------------------------------------
# 2. 删除高基数类别列 (City/Profession) & 职场相关列
# -----------------------------------------------------------
# 既然我们专注于“学生抑郁”，删除 City 和 Profession 是合理的
# 删除 Job Satisfaction 和 Work Pressure 是非常正确的决定（排除职场噪音）
cols_to_drop = ['City', 'Profession', 'Job Satisfaction', 'Work Pressure']
# 注意：我在里面加了 'Degree'，通常 Degree 对于行为模型噪音较大，可视情况保留
existing_cols_to_drop = [c for c in cols_to_drop if c in df.columns]
df = df.drop(columns=existing_cols_to_drop)
print(f"✅ 已删除无关/职场特征: {existing_cols_to_drop}")

# -----------------------------------------------------------
# 3. 处理结构性异常值 (Structural Outliers / Hidden Missing Values)
# -----------------------------------------------------------
# 这是针对你刚才发现的 "0" 值的处理逻辑
rows_before = df.shape[0]

# 3.1 清洗 CGPA
# 逻辑：CGPA 为 0 通常是无效数据（除非是刚入学还没考试，但在预测任务中这类样本无意义）
if 'CGPA' in df.columns:
    df = df[df['CGPA'] > 0]

# 3.2 清洗 Academic Pressure 和 Study Satisfaction
# 逻辑：如果是纯学生数据，这两项为 0 极大概率是缺失值的占位符
# 如果你希望保留 0 代表“无压力”，可以注释掉下面这两行
if 'Academic Pressure' in df.columns:
    df = df[df['Academic Pressure'] > 0]
if 'Study Satisfaction' in df.columns:
    df = df[df['Study Satisfaction'] > 0]

rows_after = df.shape[0]
print(f"✅ 已过滤异常值 (CGPA/Pressure=0 的行). 删除行数: {rows_before - rows_after}")

# -----------------------------------------------------------
# 4. 处理常规缺失值 (Missing Values)
# -----------------------------------------------------------
# 4.1 处理 Financial Stress (如果只有几行缺失，直接删)
if 'Financial Stress' in df.columns:
    df = df.dropna(subset=['Financial Stress'])

# 4.2 Sleep Duration 缺失值填充
# 注意：先填充再做映射
if 'Sleep Duration' in df.columns:
    null_count = df['Sleep Duration'].isnull().sum()
    if null_count > 0:
        mode_val = df['Sleep Duration'].mode()[0]
        df['Sleep Duration'] = df['Sleep Duration'].fillna(mode_val)
        print(f"✅ Sleep Duration: 填充了 {null_count} 个缺失值 (使用众数: {mode_val})")




print("-" * 30)

# 最后检查一眼数据类型，确保没有 object 了
print("\n--- 清洗后数据概览 ---")
print(f"数据形状: {df.shape}")
print(df.info())


--- 开始数据清洗 ---
初始数据形状: (27901, 18)
✅ 已删除 ID 列
✅ 已删除无关/职场特征: ['City', 'Profession', 'Job Satisfaction', 'Work Pressure']
✅ 已过滤异常值 (CGPA/Pressure=0 的行). 删除行数: 14
------------------------------

--- 清洗后数据概览 ---
数据形状: (27884, 13)
<class 'pandas.core.frame.DataFrame'>
Index: 27884 entries, 0 to 27900
Data columns (total 13 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Gender                                 27884 non-null  object 
 1   Age                                    27884 non-null  float64
 2   Academic Pressure                      27884 non-null  float64
 3   CGPA                                   27884 non-null  float64
 4   Study Satisfaction                     27884 non-null  float64
 5   Sleep Duration                         27884 non-null  object 
 6   Dietary Habits                         27884 non-null  object 
 7   Degree                                 27884 non-nul

In [4]:
# ==============================================================================
# Step 2.5: 重命名特征列 (Rename Columns)
# ==============================================================================

df = df.rename(columns={
    "Gender": "Gender",
    "Age": "Age",
    "Academic Pressure": "AcadPressure",
    "CGPA": "CGPA",
    "Study Satisfaction": "StudySat",
    "Sleep Duration": "SleepDur",
    "Dietary Habits": "DietHabit",
    "Degree": "Degree",
    "Have you ever had suicidal thoughts ?": "Suicidal",
    "Work/Study Hours": "StudyHours",
    "Financial Stress": "FinStress",
    "Family History of Mental Illness": "FamMentalHx",
    "Depression": "Depression"
})
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 27884 entries, 0 to 27900
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Gender        27884 non-null  object 
 1   Age           27884 non-null  float64
 2   AcadPressure  27884 non-null  float64
 3   CGPA          27884 non-null  float64
 4   StudySat      27884 non-null  float64
 5   SleepDur      27884 non-null  object 
 6   DietHabit     27884 non-null  object 
 7   Degree        27884 non-null  object 
 8   Suicidal      27884 non-null  object 
 9   StudyHours    27884 non-null  float64
 10  FinStress     27884 non-null  float64
 11  FamMentalHx   27884 non-null  object 
 12  Depression    27884 non-null  int64  
dtypes: float64(6), int64(1), object(6)
memory usage: 3.0+ MB
None


**Fig. Feature Comparison Radar**

In [ ]:
import sys
import importlib # 专门用来刷新模块的工具

# 1. 确保路径存在
sys.path.append('/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/src')

# 2. 导入模块
import theme
import visualizer

# 1. 为了保险，重新读取原始数据 (或者确保 df 是最原始的版本)
# 假设你的文件路径是这个
df_raw_for_radar = pd.read_csv('/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/raw/Student_Depression_Dataset.csv') # 请替换为你的真实路径

# 2. 【关键】清理一下列名的空格 (防止 'Sleep Duration ' 这种情况)
df_raw_for_radar.columns = df_raw_for_radar.columns.str.strip()

# 3. 再次确认一下列都在
print("检查关键列是否存在:", 'Sleep Duration' in df_raw_for_radar.columns)
# 如果这里打印 True，下面就一定能跑通

# 4. 绘图
from visualizer import plot_radar_chart
import plotly.io as pio
pio.renderers.default = "colab" # 确保在 Colab 显示

fig = plot_radar_chart(df_raw_for_radar) # <--- 传入原始数据
fig.show()

# **3.FEATURE ENCODING**

In [5]:
# ==============================================================================
# Step 3: 特征编码 (Feature Encoding) - 修复版
# ==============================================================================
print("\n--- 开始特征编码 (修复版) ---")

import pandas as pd
import numpy as np

# 假设 df 是你当前的 DataFrame (13列)
print("--- 开始处理 ---")

# ======================================================
# 1. 字符串清洗 (防止空格干扰)
# ======================================================
obj_cols = df.select_dtypes(include=['object']).columns
for col in obj_cols:
    df[col] = df[col].str.strip()

# ======================================================
# 2. Gender 手动映射 (保留 Gender 列名)
# ======================================================
# 目标：保持列名为 'Gender'，内容变成 1 (Male) 和 0 (Female)
print("处理 Gender...")
gender_map = {'Male': 1, 'Female': 0, 'male': 1, 'female': 0}
df['Gender'] = df['Gender'].map(gender_map).fillna(0).astype(int)

# ======================================================
# 3. 二值变量映射 (Suicidal, FamMentalHx)
# ======================================================
print("处理二值变量...")
binary_map = {'Yes': 1, 'No': 0, 'yes': 1, 'no': 0}
cols_binary = ['Suicidal', 'FamMentalHx']

for col in cols_binary:
    df[col] = df[col].map(binary_map).fillna(0).astype(int)

# ======================================================
# 4. 多分类变量 One-Hot (全量保留)
# ======================================================
# 包含: DietHabit, SleepDur, Degree
cols_to_onehot = ['DietHabit', 'SleepDur', 'Degree']
print(f"执行 One-Hot (保留所有类别): {cols_to_onehot}")

# drop_first=False: 这样 Healthy, Moderate, Unhealthy 都会分别生成一列
df = pd.get_dummies(df, columns=cols_to_onehot, drop_first=False)

# 将 bool 转为 int (1/0)
bool_cols = df.select_dtypes(include=['bool']).columns
df[bool_cols] = df[bool_cols].astype(int)

# ======================================================
# 5. 打印最终 Info
# ======================================================
print("\n" + "="*30)
print("🎉 处理完成！最终数据概览:")
print("="*30)
df.info()

# 验证一下 DietHabit 是否保留了 3 列
diet_cols = [c for c in df.columns if 'DietHabit' in c]
print(f"\n检查 DietHabit 列: {diet_cols}")


--- 开始特征编码 (修复版) ---
--- 开始处理 ---
处理 Gender...
处理二值变量...
执行 One-Hot (保留所有类别): ['DietHabit', 'SleepDur', 'Degree']

🎉 处理完成！最终数据概览:
<class 'pandas.core.frame.DataFrame'>
Index: 27884 entries, 0 to 27900
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Gender                      27884 non-null  int64  
 1   Age                         27884 non-null  float64
 2   AcadPressure                27884 non-null  float64
 3   CGPA                        27884 non-null  float64
 4   StudySat                    27884 non-null  float64
 5   Suicidal                    27884 non-null  int64  
 6   StudyHours                  27884 non-null  float64
 7   FinStress                   27884 non-null  float64
 8   FamMentalHx                 27884 non-null  int64  
 9   Depression                  27884 non-null  int64  
 10  DietHabit_Healthy           27884 non-null  int64  
 11  DietHabit_Moderate 

In [6]:
# ==============================================================================
# 6. 保存已编码 & 清洗后的数据 (Processed Dataset)
# ==============================================================================
import os

processed_dir = "/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/processed"
os.makedirs(processed_dir, exist_ok=True)

processed_path = os.path.join(processed_dir, "student_depression_encoded.csv")

df.to_csv(processed_path, index=False, encoding="utf-8")

print(f"\n✅ 已将编码后的数据保存至:\n{processed_path}")



✅ 已将编码后的数据保存至:
/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/processed/student_depression_encoded.csv


In [7]:
with open(os.path.join(processed_dir, "feature_columns.txt"), "w") as f:
    for col in df.columns:
        f.write(col + "\n")


**Fig. Feature Correlation**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import scipy.cluster.hierarchy as sch

# ==========================================
# 1. 数据准备 & 标签映射 (使用 One-Hot 后的精确列名)
# ==========================================
df_viz = df.copy() # 使用经过 One-Hot 编码的 DataFrame 副本

# 1.1 标签重命名 (从 One-Hot 编码名映射到展示名)
rename_map = {
    'Depression': 'Depression',
    'Suicidal': 'Suicidal Thoughts',      # 【修复】使用原始列名 'Suicidal'
    'FamMentalHx': 'Family History',
    'FinStress': 'Financial Stress',
    'Gender': 'Gender',                      # 【修复】使用原始列名 'Gender'
    'Age': 'Age',
    'AcadPressure': 'Academic Pressure',
    'StudySat': 'Study Satisfaction',
    'CGPA': 'CGPA',
    'StudyHours': 'Study Hours',

    # Diet (使用您提供的精确列名)
    'DietHabit_Healthy': 'Diet: Healthy',
    'DietHabit_Moderate': 'Diet: Moderate',
    'DietHabit_Unhealthy': 'Diet: Unhealthy',
    'DietHabit_Others': 'Diet: Others',

    # Sleep (使用您提供的精确列名)
    'SleepDur_Less than 5 hours': 'Sleep: <5h',
    'SleepDur_5-6 hours': 'Sleep: 5-6h',
    'SleepDur_7-8 hours': 'Sleep: 7-8h',
    'SleepDur_More than 8 hours': 'Sleep: >8h',
    'SleepDur_Others': 'Sleep: Others',

    # Degree (只保留 Class 12，并将其改名)
    'Degree_Class 12': 'Degree: Class 12',
}

# 1.2 过滤列：只保留我们关心的 One-Hot 列 + 数值列 + 重命名
cols_to_keep = list(rename_map.keys())
df_viz = df_viz[cols_to_keep].copy()

# 1.3 应用重命名
df_viz = df_viz.rename(columns=rename_map)

# 1.4 计算相关系数
corr = df_viz.corr()
corr = corr.dropna(how='all', axis=0).dropna(how='all', axis=1)

# ==========================================
# 2. 聚类排序 (保持不变)
# ==========================================
d = sch.distance.pdist(corr)
L = sch.linkage(d, method='ward')
dendro = sch.dendrogram(L, no_plot=True)
reordered_index = dendro['leaves']
corr = corr.iloc[reordered_index, reordered_index]

# ==========================================
# 3. 颜色定义 (保持不变)
# ==========================================
color_lime_final = "#8FD94D"
color_sakura_bright = "#FF5588"

colors_bright_final = [
    color_lime_final,   # [最负]
    "#D9F0C0",          # [微负]
    "#FFFFFF",          # [极窄中点]
    "#FFC4D6",          # [微正]
    color_sakura_bright # [最正]
]
cmap_bright = LinearSegmentedColormap.from_list("SakuraBright", colors_bright_final, N=256)

# ==========================================
# 4. 高清绘图 (左大右小布局)
# ==========================================
plt.rcParams['figure.dpi'] = 300
fig, axes = plt.subplots(1, 2, figsize=(18, 10), gridspec_kw={'width_ratios': [1.8, 1]})

# --- 左图：聚类热力图 ---
ax_matrix = axes[0]
sns.heatmap(
    corr,
    cmap=cmap_bright,
    annot=True,
    fmt=".2f",
    annot_kws={"size": 7, "color": "black"},
    cbar=False,
    center=-0.05, vmin=-1.0, vmax=0.9,
    linewidths=0,
    square=True,
    ax=ax_matrix
)
# 标题居中 + 留白
ax_matrix.set_title("A. Feature Clustering Matrix (Annotated)", fontweight='bold', pad=30, loc='center', fontsize=14)
ax_matrix.tick_params(axis='both', which='both', length=0)
plt.setp(ax_matrix.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# --- 右图：焦点棒棒糖 ---
ax_lollipop = axes[1]
target_col = 'Depression'
if target_col in corr.columns:
    lolli_data = corr[target_col].drop(target_col).sort_values(ascending=True)

    # 颜色分配
    my_colors = [color_sakura_bright if val > 0 else color_lime_final for val in lolli_data.values]

    ax_lollipop.hlines(y=lolli_data.index, xmin=0, xmax=lolli_data.values, color=my_colors, alpha=1, linewidth=2.5)
    ax_lollipop.scatter(lolli_data.values, lolli_data.index, color=my_colors, s=100, alpha=1, zorder=3)

    for i, (val, label) in enumerate(zip(lolli_data.values, lolli_data.index)):
        offset = 0.03 if val >= 0 else -0.03
        ha_pos = 'left' if val >= 0 else 'right'
        ax_lollipop.text(val + offset, i, f"{val:.2f}", va='center', ha=ha_pos,
                         fontsize=9, color='black', fontweight='bold')

    ax_lollipop.axvline(0, color='grey', linestyle=':', linewidth=1)
    ax_lollipop.set_xlim(-0.6, 0.6)
    ax_lollipop.set_title("B. Correlation with Depression", fontweight='bold', pad=20, loc='left', fontsize=14)
    ax_lollipop.set_xlabel("Correlation Coefficient (r)")

    sns.despine(ax=ax_lollipop, left=True, bottom=False, top=True, right=True)
    ax_lollipop.tick_params(axis='y', which='both', length=0)
    ax_lollipop.grid(axis='both', linestyle=':', alpha=0.6)

# 5. 全局图例
cbar_ax = fig.add_axes([0.35, 0.02, 0.3, 0.02])
norm = plt.Normalize(-0.8, 1)
sm = plt.cm.ScalarMappable(cmap=cmap_bright, norm=norm)
sm.set_array([])
cb = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal', ticks=[-1, 0, 1])
cb.set_label('Correlation Strength (Green=Protective, Pink=Risk)', fontsize=10, labelpad=10)

# 两侧宽敞留白
plt.subplots_adjust(top=0.85, bottom=0.150, wspace=0.4, left=0.3, right=1.0)

plt.show()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27884 entries, 0 to 27900
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Gender                      27884 non-null  int64  
 1   Age                         27884 non-null  float64
 2   AcadPressure                27884 non-null  float64
 3   CGPA                        27884 non-null  float64
 4   StudySat                    27884 non-null  float64
 5   Suicidal                    27884 non-null  int64  
 6   StudyHours                  27884 non-null  float64
 7   FinStress                   27884 non-null  float64
 8   FamMentalHx                 27884 non-null  int64  
 9   Depression                  27884 non-null  int64  
 10  DietHabit_Healthy           27884 non-null  int64  
 11  DietHabit_Moderate          27884 non-null  int64  
 12  DietHabit_Others            27884 non-null  int64  
 13  DietHabit_Unhealthy         27884 no

**Fig. Feature Attribute**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
from matplotlib.colors import LinearSegmentedColormap

# ==========================================
# 1. 准备数据 (固定 Objectivity 和 Correlation)
# ==========================================
target_col = 'Depression'
y_data = corr[target_col].drop(target_col, errors='ignore')

# X轴分数: Objectivity Score (客观性：1 = 感受, 5 = 事实/档案)
# C轴颜色: Deployability Score (部署容易程度：5 = 容易/低成本, 1 = 困难/高成本)

feature_scores_optimized = {
    # 评分逻辑不变，但我们现在称之为 Deployability Score
    # --- Clinical / Survey (最难部署) ---
    'Suicidal Thoughts': {'obj': 1.0, 'dep_score': 1.0},
    'Family History':    {'obj': 5.0, 'dep_score': 1.8},
    'Financial Stress':  {'obj': 1.6, 'dep_score': 2.0},
    'Study Satisfaction':{'obj': 1.8, 'dep_score': 2.2},
    'Academic Pressure': {'obj': 2.0, 'dep_score': 2.2},

    # --- Behavioral / Sensor (最易部署) ---
    'Diet: Unhealthy':   {'obj': 3.2, 'dep_score': 4.0},
    'Study Hours':       {'obj': 4.0, 'dep_score': 4.5},
    'Sleep: <5h':        {'obj': 4.2, 'dep_score': 4.8},
    'Sleep: 5-6h':       {'obj': 4.2, 'dep_score': 4.8},
    'Sleep: 7-8h':       {'obj': 4.2, 'dep_score': 4.8},
    'Sleep: >8h':        {'obj': 4.2, 'dep_score': 4.8},
    'Sleep: Others':     {'obj': 4.2, 'dep_score': 4.8},
    'Diet: Moderate':    {'obj': 3.2, 'dep_score': 4.0},
    'Diet: Healthy':     {'obj': 3.2, 'dep_score': 4.0},
    'Diet: Others':      {'obj': 3.2, 'dep_score': 4.0},

    # --- Demographic / Record (客观，但中等摩擦) ---
    'CGPA':              {'obj': 5.0, 'dep_score': 3.0},
    'Degree: Class 12': {'obj': 5.0, 'dep_score': 3.2},
    'Gender':            {'obj': 5.0, 'dep_score': 3.5},
    'Age':               {'obj': 5.0, 'dep_score': 3.5},
}

# 构建绘图 DataFrame
df_quad = pd.DataFrame({'Correlation': y_data})
df_quad['Objectivity'] = df_quad.index.map(lambda x: feature_scores_optimized.get(x, {}).get('obj', np.nan))
df_quad['Deployability_Score'] = df_quad.index.map(lambda x: feature_scores_optimized.get(x, {}).get('dep_score', np.nan))
df_quad = df_quad.dropna(subset=['Objectivity', 'Deployability_Score'])


# ==========================================
# 3. 颜色定义 (用 Deployability Score 决定颜色)
# ==========================================

# 【核心修改】替换颜色渐变，引入米金色中点
colors_gradient_theme = [
    (0.0, '#FF5588'),  # 0.0 (最低分/难部署) = 主题粉
    (0.5, '#F0D49C'),  # 0.5 (中点) = 柔和米金色 (协调色)
    (1.0, '#8FD94D')   # 1.0 (最高分/易部署) = 主题绿
]
cmap_deploy = LinearSegmentedColormap.from_list("ThemeFeasibility", colors_gradient_theme)

score_min = df_quad['Deployability_Score'].min()
score_max = df_quad['Deployability_Score'].max()
df_quad['Color_Value'] = (df_quad['Deployability_Score'] - score_min) / (score_max - score_min)

# ==========================================
# 4. 绘图 (三维度展示)
# ==========================================
plt.figure(figsize=(12, 10), dpi=300)

plt.axhline(0, color='black', linewidth=1.5, linestyle='-')
plt.axvline(3.5, color='grey', linewidth=1, linestyle='--', alpha=0.5)

scatter = plt.scatter(
    x=df_quad['Objectivity'],
    y='Correlation',
    data=df_quad,
    c='Color_Value',
    cmap=cmap_deploy,
    s=300,
    edgecolor='black',
    linewidth=1,
    alpha=0.9,
    zorder=10
)

# --- 添加标签避让策略 (只移动文字标签) ---
manual_offsets = {
    # CGPA: 左上 (x_offset < 0, y_offset > 0)
    'CGPA': {'x_offset': -0.08, 'y_offset': 0.03, 'ha': 'right', 'va': 'bottom'},

    # Gender: 左中
    'Gender': {'x_offset': -0.08, 'y_offset': -0.02, 'ha': 'right', 'va': 'center'},

    # Family History: 左下
    'Family History': {'x_offset': -0.05, 'y_offset': -0.03, 'ha': 'right', 'va': 'top'},
}

for idx, row in df_quad.iterrows():
    # 默认偏移
    y_offset = 0.02 if row['Correlation'] > 0 else -0.02
    x_offset = 0
    va_align = 'bottom' if row['Correlation'] > 0 else 'top'
    ha_align = 'center'

    # 应用手动优化
    if idx in manual_offsets:
        offset_data = manual_offsets[idx]
        x_offset = offset_data.get('x_offset', x_offset)
        y_offset = offset_data.get('y_offset', y_offset)
        va_align = offset_data.get('va', va_align)
        ha_align = offset_data.get('ha', ha_align)

    clean_label = idx.replace(': ', ':')

    plt.text(
        row['Objectivity'] + x_offset,
        row['Correlation'] + y_offset,
        clean_label,
        fontsize=9,
        color='#333333',
        fontweight='semibold',
        ha=ha_align,
        va=va_align
    )

# ==========================================
# 5. 区域标注 (纯属性描述)
# ==========================================
plt.title("Feature Attribute Matrix: Predictive Impact vs. Data Objectivity", fontsize=18, fontweight='bold', pad=20)
plt.ylabel("Predictive Impact (Correlation with Depression)", fontsize=12, fontweight='bold')
plt.xlabel("Data Objectivity (Subjective $\\rightarrow$ Factual)", fontsize=14, fontweight='bold', labelpad=15)

plt.xticks([1.5, 3.5, 5.0],
           ['Subjective Domain', 'Behavioral/Sensor', 'Archival/Static'])
plt.xlim(0.8, 5.2)
y_max = df_quad['Correlation'].max()
y_min = df_quad['Correlation'].min()
plt.ylim(y_min - 0.1, y_max + 0.15)

# --- 添加颜色条 (Color Bar) 作为第三维度图例 ---
cbar = plt.colorbar(scatter, ax=plt.gca(), orientation='vertical', pad=0.02, shrink=0.7)
cbar.set_label("Deployment Feasibility (Red = Hard/Costly, Green = Easy/Automated)", fontsize=10)


plt.grid(True, linestyle=':', alpha=0.3)
plt.tight_layout()

plt.savefig('feature_attribute_matrix_final_v13_golden_midpoint.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27884 entries, 0 to 27900
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Gender                      27884 non-null  int64  
 1   Age                         27884 non-null  float64
 2   AcadPressure                27884 non-null  float64
 3   CGPA                        27884 non-null  float64
 4   StudySat                    27884 non-null  float64
 5   Suicidal                    27884 non-null  int64  
 6   StudyHours                  27884 non-null  float64
 7   FinStress                   27884 non-null  float64
 8   FamMentalHx                 27884 non-null  int64  
 9   Depression                  27884 non-null  int64  
 10  DietHabit_Healthy           27884 non-null  int64  
 11  DietHabit_Moderate          27884 non-null  int64  
 12  DietHabit_Others            27884 non-null  int64  
 13  DietHabit_Unhealthy         27884 no

# **4. DATA SPLIT & SCALING**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np

# 假设 df 变量包含您提供的 Dtype 信息所示的 One-Hot 编码后的最终 DataFrame

# 定义所有需要缩放的数值列 (连续特征 + 量表特征)
features_to_scale = [
    'Age', 'CGPA', 'StudyHours',
    'AcadPressure', 'StudySat', 'FinStress'
]

# 提取 X (特征) 和 y (目标)
# 【安全措施 1】对 X 进行深度复制，确保后续操作不会修改原始 df
X = df.drop(columns=['Depression']).copy()
y = df['Depression'].copy()

# 确保 features_to_scale 列表中的列在 X 中存在
features_to_scale = [col for col in features_to_scale if col in X.columns]


# ==========================================
# 1. 拆分：测试集 (Test Set, 15%)
# ==========================================
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y,
    test_size=0.15,
    random_state=42,
    stratify=y
)

# ==========================================
# 2. 拆分：训练集 (Train Set, 70%) 和 验证集 (Validation Set, 15%)
# ==========================================
val_size_relative = 0.15 / 0.85

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=val_size_relative,
    random_state=42,
    stratify=y_train_val
)


# ==========================================
# 3. 特征标准化 (Standard Scaling)
# ==========================================
scaler = StandardScaler()

# 核心：只在 X_train 上 fit (学习均值和标准差)
# 【安全措施 2】 使用 .loc 和赋值操作，确保 Standard Scaler 的结果赋值给正确的副本
X_train.loc[:, features_to_scale] = scaler.fit_transform(X_train[features_to_scale])

# 用训练集学习到的参数 transform 验证集和测试集
X_val.loc[:, features_to_scale] = scaler.transform(X_val[features_to_scale])
X_test.loc[:, features_to_scale] = scaler.transform(X_test[features_to_scale])

print("✅ 特征标准化完成 (StandardScaler)")


# ==========================================
# 4. 处理类别不平衡 (SMOTE) - 仅在训练集上进行
# ==========================================
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"\n--- 数据集分配情况 ---")
print(f"训练集大小 (~70%): {len(X_train)}")
print(f"验证集大小 (~15%): {len(X_val)}")
print(f"测试集大小 (15%): {len(X_test)}")
print(f"SMOTE处理后训练集总样本数: {len(X_train_resampled)}")
print("✅ 数据准备流程已完成，原始 df 保持不变。")


# 5. 最终数据输出 (保留不变)
data_for_modeling = {
    'X_train_smote': X_train_resampled,
    'y_train_smote': y_train_resampled,
    'X_val': X_val,
    'y_val': y_val,
    'X_test': X_test,
    'y_test': y_test,
    'scaler': scaler
}

✅ 特征标准化完成 (StandardScaler)

--- 数据集分配情况 ---
训练集大小 (~70%): 19518
验证集大小 (~15%): 4183
测试集大小 (15%): 4183
SMOTE处理后训练集总样本数: 22858
✅ 数据准备流程已完成，原始 df 保持不变。


In [ ]:
# ==========================================
# 6. 保存 StandardScaler（仅基于 X_train 拟合）
# ==========================================
import joblib
import os

scaler_dir = "/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/src"
scaler_path = os.path.join(scaler_dir, "standard_scaler_train_only.pkl")

joblib.dump(scaler, scaler_path)

print(f"✅ StandardScaler 已保存至: {scaler_path}")

✅ StandardScaler 已保存至: /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/src/standard_scaler_train_only.pkl


# **5.SAVE PROCESSED DATA**



In [ ]:
import os
import pandas as pd
import numpy as np

# 假设 data_for_modeling 字典在内存中已存在，包含以下键：
# 'X_train_smote', 'y_train_smote', 'X_val', 'y_val', 'X_test', 'y_test'

# 定义您的目标文件夹路径
SAVE_PATH = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/processed'

# 确保目标路径存在
os.makedirs(SAVE_PATH, exist_ok=True)

print("--- 开始保存处理后的数据集到 Google Drive ---")

# 1. 训练集 (SMOTE后)
data_for_modeling['X_train_smote'].to_csv(os.path.join(SAVE_PATH, 'X_train_smote.csv'), index=False)
data_for_modeling['y_train_smote'].to_csv(os.path.join(SAVE_PATH, 'y_train_smote.csv'), index=False, header=True)
print(f"✅ 训练集保存成功: X_train_smote.csv / y_train_smote.csv")

# 2. 验证集
data_for_modeling['X_val'].to_csv(os.path.join(SAVE_PATH, 'X_val.csv'), index=False)
data_for_modeling['y_val'].to_csv(os.path.join(SAVE_PATH, 'y_val.csv'), index=False, header=True)
print(f"✅ 验证集保存成功: X_val.csv / y_val.csv")

# 3. 测试集
data_for_modeling['X_test'].to_csv(os.path.join(SAVE_PATH, 'X_test.csv'), index=False)
data_for_modeling['y_test'].to_csv(os.path.join(SAVE_PATH, 'y_test.csv'), index=False, header=True)
print(f"✅ 测试集保存成功: X_test.csv / y_test.csv")

print("\n🎉 所有数据集已成功保存到您的 Google Drive 目标文件夹。")

--- 开始保存处理后的数据集到 Google Drive ---
✅ 训练集保存成功: X_train_smote.csv / y_train_smote.csv
✅ 验证集保存成功: X_val.csv / y_val.csv
✅ 测试集保存成功: X_test.csv / y_test.csv

🎉 所有数据集已成功保存到您的 Google Drive 目标文件夹。
